# Glacier advance and retreat

Goals of this notebook:

- understand the concept of the equilibrium line altitude (ELA)
- understand the influence of glacier mass balance on the ELA
- be able to explain glacier advance and retreat in response to a change in the ELA

In [ ]:
import oggm
from oggm import cfg
import oggm_edu as edu
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import numpy as np
from IPython.display import Image
from functools import partial

In [ ]:
# set default font size in plots
plt.rc('font', size=14)

## Equilibrium line altitude

Recall the definition of glacier mass balance from the [accumulation and ablation notebook](accumulation_and_ablation.ipynb),

$$\dot{m} = \text{accumulation} + \text{ablation}.$$

The rates of accumulation and ablation processes, summed over the glacier and over time, determine the *glacier mass balance* $\dot{m}$, the change in total mass of snow and ice. Since accumulation and ablation generally vary with height, also the glacier mass balance is a function of elevation,

$$\dot{m}(z) = \text{accumulation}(z) + \text{ablation}(z).$$

The altitude where $\dot{m}(z) = 0$ is called the *equilibrium line altitude*, short ELA. Hence, the ELA is the altitude where accumulation processes and ablation processes balance each other - in theory. However, in reality the ELA does not exactly exist and can only be approximated.

In the introduction on the OGGM-Edu [website](http://edu.oggm.org/en/latest/glacier_basics.html), the ELA of a glacier in equilibrium is shown.

In [ ]:
Image(url='https://raw.githubusercontent.com/OGGM/glacier-graphics/master/glacier_intro/png/glacier_07.png', width=600)

## Glacier advance and retreat with OGGM

In [ ]:
cfg.initialize()

First, as always, we define a linear bedrock profile:

In [ ]:
# define horizontal resolution of the model:
# nx: number of grid points
# map_dx: grid point spacing in meters
nx = 200
map_dx = 200

In [ ]:
# define glacier top and bottom altitudes in meters
top = 5000 
bottom = 0

In [ ]:
# create a linear bedrock profile from top to bottom
bed_h, surface_h = edu.define_linear_bed(top, bottom, nx)

In [ ]:
# calculate the distance from the top to the bottom of the glacier in km
distance_along_glacier = edu.distance_along_glacier(nx, map_dx)

Then we define the bedrock shape, the glacier width and the mass balance model.

In [ ]:
# glacier width at the top of the accumulation area: m
ACCW = 1000

In [ ]:
# glacier width at the equilibrium line altitude: m
ELAW = 500

In [ ]:
# fraction of vertical grid points occupied by accumulation area
NZ = 1 / 3

In [ ]:
# mass balance gradient with respect to elevation in mm w.e. m^-1 yr^-1
mb_grad = 7

Finally, we let the glacier evolve until reaching an equilibrium state. To speed this part up, use the helper function provided in the oggm_edu package:

In [ ]:
# initialize the model glacier with a linear mass balance profile
# this pseudo-3D plot shows the glacier geometry
model = edu.linear_mb_equilibrium(bed_h, surface_h, ACCW, ELAW, NZ, mb_grad, nx, map_dx)

In [ ]:
# the glacier surface in equilibrium
initial = model.fls[-1].surface_h

In [ ]:
# the glacier width along the flowline
mwidths = model.fls[-1].widths

In [ ]:
# the equilibrium line altitude
ela = model.mb_model.get_ela()

Now, we use a linear accumulation and ablation function from the oggm-edu package:

In [ ]:
# accumulation and ablation balance each other
acc, acc_0 = edu.linear_accumulation(mb_grad, ela, initial, bed_h, mwidths)
abl, abl_0 = edu.linear_ablation(mb_grad, ela, initial, bed_h, mwidths, acc_0)

``acc_0`` and ``abl_0`` are the accumulation and the ablation at the ELA, respectively - by construction, they should be equal:

In [ ]:
print('Mass balance at the ELA: {:.2f} m w.e. yr^-1'.format(float(acc_0 - abl_0)))

Now, we can define the glacier surface after accumulation and ablation, respectively.

In [ ]:
# accumulation and ablation surfaces
acc_sfc = initial + acc
abl_sfc = initial + abl

The mass balance is then just the sum of accumulation and ablation:

In [ ]:
# net mass balance m w.e yr^-1
mb_we = acc + abl

Near the terminus ablation may totally remove the ice, hence, we need to correct "negative" ice thickness to the glacier bed:

In [ ]:
# mass balance surface corrected to the bed where ice thickness is negative
mb_sfc = edu.correct_to_bed(bed_h, initial + mb_we)

The next figure shows te glacier surface after applying the net mass balance.

In [ ]:
# plot the model glacier
fig, ax = plt.subplots(1, 1, figsize=(16, 9))
edu.intro_glacier_plot(ax, distance_along_glacier, bed_h, initial, [mb_sfc], ['mass balance'], ela=[ela], plot_ela=True)

Now we have set the scene to model glacier advance and retreat.

## Advance

More accumulation and/or less ablation leads to a decrease of the ELA and therefore to an increase of the accumulation area and a decrease of the ablation area, as shown in this graph:

In [ ]:
Image(url='https://raw.githubusercontent.com/OGGM/glacier-graphics/master/glacier_intro/png/glacier_08.png', width=600)

To simulate a glacier advance, we will use the same glacier as before, but move the ELA downglacier:

In [ ]:
# number of vertical grid points the ELA is shifted downglacier
downglacier = 10

In [ ]:
# run the model until the glacier reaches an equilibrium state with its decreased ELA
advance = edu.linear_mb_equilibrium(bed_h, surface_h, ACCW, ELAW, NZ, mb_grad, nx, map_dx, idx=downglacier, advance=True, plot=False)

In [ ]:
# decreased ELA
ela_adv = advance.mb_model.get_ela()

In [ ]:
print(*['{} ELA: {:.2f} m'.format(s, e) for s, e in zip(['Initial', 'Advance'], [ela, ela_adv])], sep='\n')

Now, we can again calculate linear accumulation, ablation and mass balance profiles, but using the decreased ELA:

In [ ]:
# accumulation and ablation balance at the decreased ELA
acc_adv, acc_0_adv = edu.linear_accumulation(mb_grad, ela_adv, initial, bed_h, mwidths)
abl_adv, abl_0_adv = edu.linear_ablation(mb_grad, ela_adv, initial, bed_h, mwidths, acc_0_adv)

In [ ]:
# net mass balance m w.e yr^-1
mb_adv = acc_adv + abl_adv

In [ ]:
# mass balance surface corrected to the bed where ice thickness is negative
mb_sfc_adv = edu.correct_to_bed(bed_h, initial + mb_adv)

The next figure shows the glacier after applying the net mass balance using the decreased ELA:

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 9))
edu.intro_glacier_plot(ax, distance_along_glacier, bed_h, initial, [mb_sfc_adv], ['Advance'], ela=[ela, ela_adv], plot_ela=True)

The increase in the accumulation area leads to a net mass gain compared to the former glacier extent. As discussed in the [accumulation and ablation notebook](accumulation_and_ablation.ipynb), mass is transported downglacier by a gravity driven ice flow obeying conservation of mass

$$\frac{\partial H}{\partial t} = \dot{m} - \nabla \cdot \vec{q}.$$

Therefore, a decrease in the ELA and the resulting net mass gain will lead to a glacier advance as a result of an increased ice flow $\vec{q}$, as indicated as thick arrow in this graph:

In [ ]:
Image(url='https://raw.githubusercontent.com/OGGM/glacier-graphics/master/glacier_intro/png/glacier_09.png', width=600)

We can verify this by plotting the glacier surface after the decrease in the ELA and comparing it to the former glacier surface:

In [ ]:
# the glacier surface in equilibriium with the decreased ELA
advance_s = advance.fls[-1].surface_h

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 9))
edu.intro_glacier_plot(ax, distance_along_glacier, bed_h, initial, [advance_s], ['Advance'], ela=[ela, ela_adv], plot_ela=True)

As expected, the glacier advanced downslope.

## Retreat

Analogously, less accumulation and/or more ablation leads to an increase in the ELA and thus to an increase in the ablation area.

In [ ]:
Image(url='https://raw.githubusercontent.com/OGGM/glacier-graphics/master/glacier_intro/png/glacier_10.png', width=600)

To simulate a glacier retreat, we will again use the same glacier, but this time move the ELA upglacier.

In [ ]:
# number of vertical grid points the ELA is shifted upglacier
upglacier = 10

In [ ]:
# run the model until the glacier reaches an equilibrium state with its decreased ELA
retreat = edu.linear_mb_equilibrium(bed_h, surface_h, ACCW, ELAW, NZ, mb_grad, nx, map_dx, idx=upglacier, retreat=True, plot=False)

In [ ]:
# decreased ELA
ela_rtr = retreat.mb_model.get_ela()

In [ ]:
print(*['{} ELA: {:.2f} m'.format(s, e) for s, e in zip(['Initial', 'Retreat'], [ela, ela_rtr])], sep='\n')

Now, we can again calculate linear accumulation, ablation and mass balance profiles, but using the increased ELA:

In [ ]:
# accumulation and ablation balance at the decreased ELA
acc_rtr, acc_0_rtr = edu.linear_accumulation(mb_grad, ela_rtr, initial, bed_h, mwidths)
abl_rtr, abl_0_rtr = edu.linear_ablation(mb_grad, ela_rtr, initial, bed_h, mwidths, acc_0_rtr)

In [ ]:
# net mass balance m w.e yr^-1
mb_rtr = acc_rtr + abl_rtr

In [ ]:
# mass balance surface corrected to the bed where ice thickness is negative
mb_sfc_rtr = edu.correct_to_bed(bed_h, initial + mb_rtr)

The next figure shows the glacier after applying the net mass balance using the increased ELA:

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 9))
edu.intro_glacier_plot(ax, distance_along_glacier, bed_h, initial, [mb_sfc_rtr], ['Retreat'], ela=[ela, ela_rtr], plot_ela=True)

The increase in the ablation area leads to a net mass loss compared to the former glacier extent. Therefore, an increase in the ELA will lead to a glacier retreat as a result of a decreased ice flow $\vec{q}$, as indicated as thin arrow in this graph:

In [ ]:
Image(url='https://raw.githubusercontent.com/OGGM/glacier-graphics/master/glacier_intro/png/glacier_11.png', width=600)

We can verify this by plotting the glacier surface after the increase in the ELA and comparing it to the former glacier surface:

In [ ]:
# the glacier surface in equilibriium with the decreased ELA
retreat_s = retreat.fls[-1].surface_h

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 9))
edu.intro_glacier_plot(ax, distance_along_glacier, bed_h, initial, [retreat_s], ['Retreat'], ela=[ela, ela_rtr], plot_ela=True)

As expected, the glacier retreats upslope.

## Take home points

- The equilibrium line altitude (ELA) is the altitude on a glacier where accumulation and ablation balance, meaning $\dot{m}(z) = 0$ at $z=$ ELA
- A decrease in the ELA leads to:
    1. an increase in the accumulation
    2. a decrease in the ablation area
    3. a net mass gain resulting in an increased ice flux downglacier
    4. a glacier advance
- An increase in the ELA leads to:
    1. a decrease in the accumulation area
    2. an increase in the ablation area
    3. a net mass loss resulting in a decreased ice flux downglacier
    4. a glacier retreat

In [ ]:
# or if you are a visual type learner
fig, ax = plt.subplots(1, 1, figsize=(16, 9))
edu.intro_glacier_plot(ax, distance_along_glacier, bed_h, initial, [advance_s, retreat_s], ['Advance', 'Retreat'], ela=[ela, ela_adv, ela_rtr], plot_ela=True)

## What's next?

[Back to table of contents](welcome.ipynb)